In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google'

#requirments

In [ ]:
!pip install tensorflow>=2.4

In [ ]:
import os

Extracting training zip file

In [ ]:
cd "/content/drive/MyDrive/BTech_Dataset_transformed/02/train"

/content/drive/MyDrive/BTech_Dataset_transformed/02/train


In [ ]:
good="good"
if not os.path.exists(good):
  os.makedirs(good)
bad="anamoly"
if not os.path.exists(bad):
  os.makedirs(bad)

#preprocessing of  BTAD Dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

# Define function to load and preprocess images
def preprocess_image(image_path, target_size=(224, 224)):
    # Load image using TensorFlow
    image = load_img(image_path, target_size=target_size)
    # Convert image to numpy array
    image_array = img_to_array(image)
    # Normalize pixel values to range [0, 1]
    image_array = image_array / 255.0
    # Flatten the image
    image_array_flat = tf.reshape(image_array, [-1])
    return image_array_flat

# Define function to load and preprocess dataset
def preprocess_dataset(dataset_dir, target_size=(224, 224)):
    # Initialize lists to store preprocessed images and labels
    preprocessed_images = []
    labels = []
    # Loop through each category in the dataset directory
    for category in os.listdir(dataset_dir):
        category_dir = os.path.join(dataset_dir, category)
        # Loop through each image in the category directory
        for image_file in os.listdir(category_dir):
            image_path = os.path.join(category_dir, image_file)
            # Preprocess the image
            preprocessed_image_flat = preprocess_image(image_path, target_size=target_size)
            # Append preprocessed image to list
            preprocessed_images.append(preprocessed_image_flat)
            # Assign label based on category
            labels.append(category)
    # Convert lists to TensorFlow tensors
    preprocessed_images = tf.convert_to_tensor(preprocessed_images)
    labels = tf.convert_to_tensor(labels)
    return preprocessed_images, labels

# Define path to MVTec dataset directory
btad_dataset_dir = "/content/drive/MyDrive/BTech_Dataset_transformed/02/train"

# Preprocess the dataset
preprocessed_images, labels = preprocess_dataset(btad_dataset_dir)

# Display shapes of preprocessed images and labels
print("Shape of preprocessed images:", preprocessed_images.shape)
print("Shape of labels:", labels.shape)

Shape of preprocessed images: (1800, 150528)
Shape of labels: (1800,)


#Model

In [ ]:
import tensorflow as tf

# Enable eager execution
tf.config.run_functions_eagerly(True)

In [ ]:
preprocessed_test_images, preprocessed_test_labels = preprocess_dataset(btad_dataset_dir)

preprocessed_images_np = preprocessed_images.numpy()
labels_np = labels.numpy()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def l2_distance(z):
    diff = tf.expand_dims(z, axis=1) - tf.expand_dims(z, axis=0)
    return tf.reduce_sum(diff ** 2, axis=-1)


class PairwiseSimilarity(layers.Layer):
    def __init__(self, sigma=1.0):
        super(PairwiseSimilarity, self).__init__()
        self.sigma = sigma

    def call(self, z):
        return tf.exp(-l2_distance(z) / self.sigma)


class ContextualSimilarity(layers.Layer):
    def __init__(self, k):
        super(ContextualSimilarity, self).__init__()
        self.k = k

    def call(self, z):
        distances = l2_distance(z)
        kth_nearst = -tf.math.top_k(-distances, k=self.k, sorted=True)[0][:, -1]
        mask = tf.cast(distances <= tf.expand_dims(kth_nearst, axis=-1), tf.float32)

        similarity = tf.matmul(mask, mask, transpose_b=True) / tf.reduce_sum(mask, axis=-1, keepdims=True)
        R = mask * tf.transpose(mask)
        similarity = tf.matmul(similarity, R, transpose_b=True) / tf.reduce_sum(R, axis=-1, keepdims=True)
        return 0.5 * (similarity + tf.transpose(similarity))

class ReConPatch(tf.keras.Model):
    def __init__(self, input_dim, embedding_dim, projection_dim, alpha, margin=0.1):
        super(ReConPatch, self).__init__()
        self.alpha = alpha
        self.margin = margin

        # Add Flatten layer
        self.flatten = tf.keras.layers.Flatten()

        # embedding & projection layers
        self.embedding = tf.keras.layers.Dense(embedding_dim)
        self.projection = tf.keras.layers.Dense(projection_dim)

        # ema version of embedding & projection layers
        self.ema_embedding = tf.keras.layers.Dense(embedding_dim, trainable=False)
        self.ema_projection = tf.keras.layers.Dense(projection_dim, trainable=False)

        # Final layer with sigmoid activation
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

        # initialize layers
        self.embedding.build((None, input_dim))
        self.projection.build((None, embedding_dim))
        self.ema_embedding.build((None, input_dim))
        self.ema_projection.build((None, embedding_dim))
        self.output_layer.build((None, projection_dim))

        # ema operator
        self.ema = tf.train.ExponentialMovingAverage(decay=0.9)
        self.update_ema()

        self.pairwise_similarity = PairwiseSimilarity(sigma=1.0)
        self.contextual_similarity = ContextualSimilarity(k=3)

    def call(self, x):
        x = self.flatten(x)  # Flatten the input data
        h = self.embedding(x)
        z = self.projection(h)
        return self.output_layer(z)

    # (rest of the class remains the same)


    def train_step(self, data):
      x, y = data  # Separate input data and labels
      h_ema = self.ema_embedding(x)
      z_ema = self.ema_projection(h_ema)

      p_sim = self.pairwise_similarity(z_ema)
      c_sim = self.contextual_similarity(z_ema)
      w = self.alpha * p_sim + (1 - self.alpha) * c_sim

      with tf.GradientTape() as tape:
          h = self.embedding(x)
          z = self.projection(h)

          # Contrastive loss
          distances = tf.sqrt(l2_distance(z) + 1e-9)
          delta = distances / tf.reduce_mean(distances, axis=-1, keepdims=True)
          rc_loss = tf.reduce_sum(tf.reduce_mean(
              w * (delta ** 2) + (1 - w) * (tf.nn.relu(self.margin - delta) ** 2),
              axis=-1
          ))

      self.optimizer.minimize(rc_loss, self.trainable_variables, tape=tape)
      self.update_ema()

      return {"rc_loss": rc_loss}


    def update_ema(self):
      self.ema.apply(self.embedding.weights + self.projection.weights)

      avg_emb_w, avg_emb_b = self.ema.average(self.embedding.weights[0]), self.ema.average(self.embedding.weights[1])
      avg_proj_w, avg_proj_b = self.ema.average(self.projection.weights[0]), self.ema.average(self.projection.weights[1])

      self.ema_embedding.set_weights([avg_emb_w, avg_emb_b])


#Split the dataset into training and validation sets
from sklearn.model_selection import train_test_split
train_images_np, val_images_np, train_labels_np, val_labels_np = train_test_split(preprocessed_images_np, labels_np, test_size=0.2, random_state=42)

#Reshape the input data
train_images = tf.convert_to_tensor(train_images_np)
val_images = tf.convert_to_tensor(val_images_np)
train_labels = tf.convert_to_tensor(train_labels_np)
val_labels = tf.convert_to_tensor(val_labels_np)

batch_size = 32

#Create TensorFlow datasets for training and validation
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(batch_size)

#data input shape
input_dim = 224 * 224 * 3

#parameters
embedding_dim = 64
projection_dim = 32
alpha = 0.5
margin = 0.1

#train model
model = ReConPatch(input_dim=input_dim, embedding_dim=embedding_dim, projection_dim=projection_dim, alpha=alpha, margin=margin)
model.compile(optimizer='adam', loss='mse')
model.fit(train_dataset, epochs=25, batch_size=batch_size)

Epoch 1/25


 1/45 [..............................] - ETA: 1:32 - rc_loss: 0.7115

 2/45 [>.............................] - ETA: 6s - rc_loss: 0.3746  

 3/45 [=>............................] - ETA: 6s - rc_loss: 0.2571

 4/45 [=>............................] - ETA: 6s - rc_loss: 0.2006

...

41/45 [==========================>...] - ETA: 0s - rc_loss: 0.0107

42/45 [===========================>..] - ETA: 0s - rc_loss: 0.0108

43/45 [===========================>..] - ETA: 0s - rc_loss: 0.0108

44/45 [============================>.] - ETA: 0s - rc_loss: 0.0108

45/45 [==============================] - 8s 177ms/step - rc_loss: 0.0108


In [ ]:
# Custom layers and model
def l2_distance(z):
    diff = tf.expand_dims(z, axis=1) - tf.expand_dims(z, axis=0)
    return tf.reduce_sum(diff ** 2, axis=-1)

class PairwiseSimilarity(tf.keras.layers.Layer):
    def __init__(self, sigma=1.0):
        super(PairwiseSimilarity, self).__init__()
        self.sigma = sigma

    def call(self, z):
        return tf.exp(-l2_distance(z) / self.sigma)

class ContextualSimilarity(tf.keras.layers.Layer):
    def __init__(self, k):
        super(ContextualSimilarity, self).__init__()
        self.k = k

    def call(self, z):
        distances = l2_distance(z)
        kth_nearst = -tf.math.top_k(-distances, k=self.k, sorted=True)[0][:, -1]
        mask = tf.cast(distances <= tf.expand_dims(kth_nearst, axis=-1), tf.float32)

        similarity = tf.matmul(mask, mask, transpose_b=True) / tf.reduce_sum(mask, axis=-1, keepdims=True)
        R = mask * tf.transpose(mask)
        similarity = tf.matmul(similarity, R, transpose_b=True) / tf.reduce_sum(R, axis=-1, keepdims=True)
        return 0.5 * (similarity + tf.transpose(similarity))

class ReConPatch(tf.keras.Model):
    def __init__(self, input_dim, embedding_dim, projection_dim, alpha, margin=0.1):
        super(ReConPatch, self).__init__()
        self.alpha = alpha
        self.margin = margin

        # Add Flatten layer
        self.flatten = tf.keras.layers.Flatten()

        # embedding & projection layers
        self.embedding = tf.keras.layers.Dense(embedding_dim)
        self.projection = tf.keras.layers.Dense(projection_dim)

        # ema version of embedding & projection layers
        self.ema_embedding = tf.keras.layers.Dense(embedding_dim, trainable=False)
        self.ema_projection = tf.keras.layers.Dense(projection_dim, trainable=False)

        # Final layer with sigmoid activation
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

        # initialize layers
        self.embedding.build((None, input_dim))
        self.projection.build((None, embedding_dim))
        self.ema_embedding.build((None, input_dim))
        self.ema_projection.build((None, embedding_dim))
        self.output_layer.build((None, projection_dim))

        # ema operator
        self.ema = tf.train.ExponentialMovingAverage(decay=0.9)
        self.update_ema()

        self.pairwise_similarity = PairwiseSimilarity(sigma=1.0)
        self.contextual_similarity = ContextualSimilarity(k=3)

    def call(self, x):
        x = self.flatten(x)  # Flatten the input data
        h = self.embedding(x)
        z = self.projection(h)
        return self.output_layer(z)

    def train_step(self, data):
        x, y = data  # Separate input data and labels
        h_ema = self.ema_embedding(x)
        z_ema = self.ema_projection(h_ema)

        p_sim = self.pairwise_similarity(z_ema)
        c_sim = self.contextual_similarity(z_ema)
        w = self.alpha * p_sim + (1 - self.alpha) * c_sim

        with tf.GradientTape() as tape:
            h = self.embedding(x)
            z = self.projection(h)

            # Contrastive loss
            distances = tf.sqrt(l2_distance(z) + 1e-9)
            delta = distances / tf.reduce_mean(distances, axis=-1, keepdims=True)
            rc_loss = tf.reduce_sum(tf.reduce_mean(
                w * (delta ** 2) + (1 - w) * (tf.nn.relu(self.margin - delta) ** 2),
                axis=-1
            ))

        self.optimizer.minimize(rc_loss, self.trainable_variables, tape=tape)
        self.update_ema()

        return {"rc_loss": rc_loss}

    def update_ema(self):
        self.ema.apply(self.embedding.weights + self.projection.weights)

        avg_emb_w, avg_emb_b = self.ema.average(self.embedding.weights[0]), self.ema.average(self.embedding.weights[1])
        avg_proj_w, avg_proj_b = self.ema.average(self.projection.weights[0]), self.ema.average(self.projection.weights[1])

        self.ema_embedding.set_weights([avg_emb_w, avg_emb_b])
        self.ema_projection.set_weights([avg_proj_w, avg_proj_b])

# Split dataset
train_images_np, val_images_np, train_labels_np, val_labels_np = train_test_split(preprocessed_images_np, labels_np, test_size=0.2, random_state=42)

train_images = tf.convert_to_tensor(train_images_np)
val_images = tf.convert_to_tensor(val_images_np)
train_labels = tf.convert_to_tensor(train_labels_np)
val_labels = tf.convert_to_tensor(val_labels_np)

batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(batch_size)

input_dim = 224 * 224 * 3
embedding_dim = 64
projection_dim = 32
alpha = 0.5
margin = 0.1

model = ReConPatch(input_dim=input_dim, embedding_dim=embedding_dim, projection_dim=projection_dim, alpha=alpha, margin=margin)
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(train_dataset, epochs=25, batch_size=batch_size)


Epoch 1/25


 1/45 [..............................] - ETA: 9s - rc_loss: 0.8625

 2/45 [>.............................] - ETA: 8s - rc_loss: 0.4480

 3/45 [=>............................] - ETA: 8s - rc_loss: 0.3057

 4/45 [=>............................] - ETA: 8s - rc_loss: 0.2369

...

41/45 [==========================>...] - ETA: 0s - rc_loss: 0.0102

42/45 [===========================>..] - ETA: 0s - rc_loss: 0.0102

43/45 [===========================>..] - ETA: 0s - rc_loss: 0.0102

44/45 [============================>.] - ETA: 0s - rc_loss: 0.0102

45/45 [==============================] - 8s 182ms/step - rc_loss: 0.0102


#Evaluation

In [ ]:
print(val_labels)
print(train_labels)

tf.Tensor(
[b'anamoly' b'good' b'good' b'good' b'good' b'good' b'good' b'good'
 b'anamoly' b'good' b'anamoly' b'good' b'good' b'good' b'good' b'good'
 b'good' b'good' b'good' b'anamoly' b'good' b'good' b'good' b'good'
 b'good' b'good' b'good' b'good' b'anamoly' b'anamoly' b'anamoly' b'good'
 b'good' b'good' b'good' b'anamoly' b'anamoly' b'good' b'good' b'good'
 b'good' b'good' b'anamoly' b'good' b'good' b'good' b'good' b'good'
 b'good' b'anamoly' b'good' b'good' b'good' b'good' b'good' b'good'
 b'good' b'good' b'good' b'anamoly' b'good' b'good' b'anamoly' b'good'
 b'anamoly' b'good' b'good' b'good' b'good' b'anamoly' b'good' b'good'
 b'good' b'good' b'good' b'anamoly' b'good' b'good' b'good' b'good'
 b'anamoly' b'good' b'good' b'good' b'good' b'anamoly' b'anamoly' b'good'
 b'good' b'good' b'good' b'good' b'good' b'anamoly' b'good' b'good'
 b'anamoly' b'good' b'good' b'good' b'good' b'good' b'anamoly' b'anamoly'
 b'anamoly' b'good' b'anamoly' b'good' b'good' b'good' b'good' b'good'
 b'g

Training loss estimation

In [ ]:
def preprocess_training_labels(labels):

    label_mapping = {"good": 1.0, "anamoly": 0.0}

    #Convert string labels to numeric format
    def map_fn(label):
        label_str = label.numpy().decode("utf-8")
        return label_mapping[label_str]

    numeric_labels = tf.map_fn(map_fn, labels, dtype=tf.float32)

    return numeric_labels

#Convert train_labels tensors to strings
train_labels_string = tf.strings.as_string(train_labels)

train_labels_numeric = preprocess_training_labels(train_labels_string)

train_loss = model.evaluate(train_images, train_labels_numeric)

print("Training Loss:", train_loss)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


45/45 [==============================] - 2s 18ms/step - loss: 0.3313
Training Loss: 0.33125001192092896


Validation loss estimation

In [ ]:
import tensorflow as tf

def preprocess_labels(labels):


    label_mapping = {"good": 1.0, "anamoly": 0.0}

    def map_fn(label):
        label_str = label.numpy().decode("utf-8")
        return label_mapping[label_str]

    numeric_labels = tf.map_fn(map_fn, labels, dtype=tf.float32)

    return numeric_labels

#Convert val_labels tensors to strings
val_labels_string = tf.strings.as_string(val_labels)

val_labels_numeric = preprocess_labels(val_labels_string)

val_loss = model.evaluate(val_images, val_labels_numeric)

print("Validation Loss:", val_loss)

3/3 [==============================] - 0s 72ms/step - loss: 899.5627
Validation Loss: 899.5626831054688


In [ ]:
print(train_loss/val_loss)

1.0019182498006063


## auroc

In [ ]:

val_preds = model.predict(val_images)

val_pred_labels = (val_preds > 0.5).astype(int)


In [ ]:
from sklearn.metrics import roc_auc_score

# Assuming val_labels_numeric contains the true binary labels (0 or 1)
val_labels_numeric = preprocess_labels(val_labels_string).numpy()

# Compute the AUROC
auroc = roc_auc_score(val_labels_numeric, val_preds)
print("Validation AUROC:", auroc)
